In [1]:
import pull_bondret_treasury
import pull_CRSP_bond_returns
import pull_he_kelly_manela_factors
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np
from misc_tools import *

In [2]:
from pathlib import Path
from settings import config

OUTPUT_DIR = Path(config("OUTPUT_DIR"))
DATA_DIR = Path(config("DATA_DIR"))

Here, we load the data and prepare it for analysis:

In [3]:
open_df = pull_bondret_treasury.load_bondret_treasury_file(data_dir=DATA_DIR)
open_df["DATE"] = pd.to_datetime(open_df["DATE"], format="%Y%m%d")
open_df['tr_return'] = open_df['tr_return'] / 100
open_df['tr_ytm_match'] = open_df['tr_ytm_match'] / 100
open_df['cusip_date'] = open_df['CUSIP'] + '_' + open_df['DATE'].dt.strftime('%Y%m%d')
open_df = open_df.sort_values(by="DATE")

crsp_df = pull_CRSP_bond_returns.load_bondret(data_dir=DATA_DIR)
crsp_df['date'] = pd.to_datetime(crsp_df['date'], format="%Y%m%d")
#crsp_df['year'] = pd.to_datetime(crsp_df['year'].astype(str), format='%Y')
crsp_df['year'] = crsp_df['year'].astype(int)
crsp_df['t_yld_pt'] = crsp_df['t_yld_pt'] / 100
crsp_df['cusip_date'] = crsp_df['cusip'] + '_' + crsp_df['date'].dt.strftime('%Y%m%d')
crsp_df = crsp_df.sort_values(by="date")
#crsp_df = crsp_df.dropna(subset=["t_yld_pt"])

In [4]:
merge_stats(crsp_df, open_df, ['cusip_date'])

union                 3.673127e+06
intersection          2.342986e+06
union-intersection    1.330141e+06
intersection/union    6.378723e-01
left                  3.634773e+06
right                 2.381340e+06
left-intersection     1.291787e+06
right-intersection    3.835400e+04
intersection/left     6.446031e-01
intersection/right    9.838939e-01
dtype: float64

In [5]:
# Merge using an outer join to keep all rows from both DataFrames.
merged = pd.merge(crsp_df, open_df, on="cusip_date", how="outer", suffixes=("", "_open"))

# Create a unified 'date' column by filling NaNs in crsp_df['date'] with open_df['DATE']
merged["date_combined"] = merged["date"].fillna(merged["DATE"])

# Similarly, create a unified 'cusip' column from crsp_df['cusip'] and open_df['CUSIP']
merged["cusip_combined"] = merged["cusip"].fillna(merged["CUSIP"])

# Drop the now redundant columns ('date', 'DATE', 'cusip', 'CUSIP')
merged.drop(columns=["date", "DATE", "cusip", "CUSIP"], inplace=True)

# Rename the combined columns to 'date' and 'cusip'
merged.rename(columns={"date_combined": "date", "cusip_combined": "cusip"}, inplace=True)

# Rearrange columns
move_columns_to_front(merged, cols=['date', 'cusip', 'cusip_date'])

# Generate yield spread and related columns, dropping NaN
merged['yield_spread'] = merged['yield'] - merged['tr_ytm_match']
merged['TTM_diff'] = merged['tmt'] - merged['tau']
merged = merged.dropna(subset=["yield_spread"])

Here, we sort each bond into deciles based on yield spread, thus generating the 10 corporate bond portfolios per Nozawa (2017) used by He, Kelly, and Manela (2017).

In [6]:
merged = merged.sort_values(by="date")
merged.reset_index(drop=True, inplace=True)
merged["decile"] = merged.groupby("date")["yield_spread"].transform(lambda x: pd.qcut(x, 10, labels=False))
merged['decile'] += 11
merged

,date,cusip,cusip_date,price_eom,tmt,yield,t_yld_pt,ret_eom,year,tr_return,tr_ytm_match,tau,yield_spread,TTM_diff,decile
0,2002-08-31,000336AE7,000336AE7_20020831,97.693000,5.836111,0.073689,0.069180,-0.008212,2002.0,0.018357,0.034521,5.756164,0.039168,0.079947,17
1,2002-08-31,638585BJ7,638585BJ7_20020831,106.300933,1.900000,0.026431,0.027552,0.014530,2002.0,0.003885,0.020833,1.873973,0.005598,0.026027,11
2,2002-08-31,345397TY9,345397TY9_20020831,97.763065,9.283333,0.075898,0.079353,-0.000826,2002.0,0.027695,0.041985,9.156164,0.033913,0.127169,17
3,2002-08-31,345397TX1,345397TX1_20020831,99.213453,4.466667,0.066999,0.072544,0.005070,2002.0,0.015621,0.030428,4.405479,0.036571,0.061187,17
4,2002-08-31,345397TT0,345397TT0_20020831,101.735068,1.575000,0.049325,0.059390,0.012401,2002.0,0.002982,0.019558,1.553425,0.029767,0.021575,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713714,2023-09-30,112585AH7,112585AH7_20230930,97.296696,1.313889,0.061879,0.061516,0.003107,2023.0,0.001252,0.053918,1.295890,0.007962,0.017998,13
1713715,2023-09-30,98419MAL4,98419MAL4_20230930,79.614457,7.441667,0.056858,0.054687,-0.029085,2023.0,-0.029979,0.046593,7.339726,0.010265,0.101941,15
1713716,2023-09-30,06054E424,06054E424_20230930,9.941727,1.436111,0.004078,NaN,0.054266,2023.0,0.002083,0.051828,1.416438,-0.047750,0.019673,11
1713717,2023-09-30,89417EAH2,89417EAH2_20230930,95.549488,17.338889,0.057602,0.055653,-0.035395,2023.0,-0.043465,0.047868,17.101370,0.009735,0.237519,15


In [7]:
# Calculate the mean monthly return (ret_eom) for each decile on each date
returns_df = merged.groupby(['date', 'decile'])['ret_eom'].mean().unstack('decile')

# Do the same for the 'tr_return' column
tr_returns_df = merged.groupby(['date', 'decile'])['tr_return'].mean().unstack('decile')

# Now compute the excess returns
excess_returns_df = returns_df - tr_returns_df

In [8]:
returns_df

decile,11,12,13,14,15,16,17,18,19,20
date,,,,,,,,,,
2002-08-31,0.033959,0.024320,0.026691,0.022861,0.017404,0.018047,0.017522,0.029266,0.054644,0.021267
2002-09-30,0.026599,0.031684,0.026219,0.025081,0.019898,0.016734,0.007456,0.008210,-0.004522,-0.038421
2002-10-31,0.016177,-0.001922,-0.001249,-0.003529,-0.007693,-0.007005,-0.012073,-0.007160,-0.003227,-0.054666
2002-11-30,0.020500,0.009440,0.014341,0.012097,0.013417,0.029168,0.119116,0.039772,0.073092,0.125202
2002-12-31,0.022325,0.026912,0.023030,0.020082,0.015668,0.017759,0.011446,0.011435,0.021061,0.022545
...,...,...,...,...,...,...,...,...,...,...
2023-05-31,0.005656,-0.006668,-0.010262,-0.015354,-0.019275,-0.018014,-0.018988,-0.017883,-0.017542,-0.012827
2023-06-30,0.017815,-0.001263,0.000125,0.002764,0.005434,0.006414,0.005545,0.006485,0.006660,0.018257
2023-07-31,0.021671,0.005160,0.004283,0.003893,0.003042,0.004062,0.005298,0.004268,0.005572,0.012643


In [9]:
test_df = pull_he_kelly_manela_factors.load_he_kelly_manela_factors(data_dir=DATA_DIR)
test_df = test_df.rename(columns={"yyyymm": "date"})
test_df["date"] = pd.to_datetime(test_df["date"], format="%Y%m") + MonthEnd(0)

us_bonds_cols = [col for col in test_df.columns if col.startswith("US_bonds_")]

group_01_10 = [col for col in us_bonds_cols if 1 <= int(col.split("_")[-1]) <= 10]
group_11_20 = [col for col in us_bonds_cols if 11 <= int(col.split("_")[-1]) <= 20]

us_tr_df = test_df[['date'] + group_01_10].copy()
us_corp_df = test_df[['date'] + group_11_20].copy()

us_tr_df = us_tr_df.dropna(axis=0, how='all', subset=group_01_10)
us_corp_df = us_corp_df.dropna(axis=0, how='all', subset=group_11_20)

In [10]:
us_corp_df.iloc[343:]

,date,US_bonds_11,US_bonds_12,US_bonds_13,US_bonds_14,US_bonds_15,US_bonds_16,US_bonds_17,US_bonds_18,US_bonds_19,US_bonds_20
391,2002-08-31,0.0155,0.0234,0.0343,0.0166,0.0163,0.0132,0.0015,0.0112,0.0046,-0.0253
392,2002-09-30,0.0228,0.0262,0.0132,0.0127,0.0103,0.0045,0.0026,-0.0006,0.0048,-0.0180
393,2002-10-31,0.0002,-0.0115,-0.0036,0.0019,0.0120,0.0052,-0.0015,0.0080,-0.0097,0.0084
394,2002-11-30,-0.0017,0.0130,0.0125,0.0240,0.0165,0.0215,0.0312,0.0339,0.0439,0.0438
395,2002-12-31,0.0201,0.0256,0.0310,0.0237,0.0192,0.0130,0.0209,0.0109,0.0025,0.0211
...,...,...,...,...,...,...,...,...,...,...,...
499,2011-08-31,0.0126,0.0274,0.0195,0.0140,-0.0033,-0.0095,-0.0122,-0.0105,-0.0101,-0.0286
500,2011-09-30,0.0036,0.0196,0.0064,0.0089,-0.0067,-0.0055,-0.0059,-0.0083,-0.0073,-0.0202
501,2011-10-31,0.0030,0.0044,0.0107,0.0122,0.0178,0.0222,0.0300,0.0301,0.0342,0.0508
502,2011-11-30,-0.0007,-0.0041,-0.0106,-0.0074,-0.0164,-0.0189,-0.0299,-0.0116,-0.0197,-0.0149


In [11]:
returns_df_reset = returns_df.reset_index()

In [12]:
returns_df_reset

decile,date,11,12,13,14,15,16,17,18,19,20
0,2002-08-31,0.033959,0.024320,0.026691,0.022861,0.017404,0.018047,0.017522,0.029266,0.054644,0.021267
1,2002-09-30,0.026599,0.031684,0.026219,0.025081,0.019898,0.016734,0.007456,0.008210,-0.004522,-0.038421
2,2002-10-31,0.016177,-0.001922,-0.001249,-0.003529,-0.007693,-0.007005,-0.012073,-0.007160,-0.003227,-0.054666
3,2002-11-30,0.020500,0.009440,0.014341,0.012097,0.013417,0.029168,0.119116,0.039772,0.073092,0.125202
4,2002-12-31,0.022325,0.026912,0.023030,0.020082,0.015668,0.017759,0.011446,0.011435,0.021061,0.022545
...,...,...,...,...,...,...,...,...,...,...,...
249,2023-05-31,0.005656,-0.006668,-0.010262,-0.015354,-0.019275,-0.018014,-0.018988,-0.017883,-0.017542,-0.012827
250,2023-06-30,0.017815,-0.001263,0.000125,0.002764,0.005434,0.006414,0.005545,0.006485,0.006660,0.018257
251,2023-07-31,0.021671,0.005160,0.004283,0.003893,0.003042,0.004062,0.005298,0.004268,0.005572,0.012643
252,2023-08-31,-0.011809,-0.001824,-0.003046,-0.005923,-0.009719,-0.010044,-0.011140,-0.011010,-0.010739,-0.007120


In [13]:
common_df = pd.merge(returns_df_reset, us_corp_df, on="date", how="inner", suffixes=('_ret', '_corp'))
r_squared_list = []
for decile in range(11, 21):
    ret_col = decile
    corp_col = "US_bonds_" + str(decile)
    
    if ret_col in common_df.columns and corp_col in common_df.columns:
        sub_df = common_df[[ret_col, corp_col]].dropna()
        if len(sub_df) > 0:
            corr = sub_df[ret_col].corr(sub_df[corp_col])
            r2 = corr ** 2
        else:
            r2 = None
    else:
        r2 = None

    r_squared_list.append({"decile": decile, "r_squared": r2})

r_squared_df = pd.DataFrame(r_squared_list)

In [14]:
r_squared_df

,decile,r_squared
0,11,0.068723
1,12,0.314371
2,13,0.789861
3,14,0.859231
4,15,0.843135
5,16,0.710314
6,17,0.454285
7,18,0.400719
8,19,0.710922
9,20,0.537141


In [15]:
excess_df_reset = excess_returns_df.reset_index()
common_df_2 = pd.merge(excess_df_reset, us_corp_df, on="date", how="inner", suffixes=('_ret', '_corp'))
r_squared_list_2 = []
for decile in range(11, 21):
    ret_col = decile
    corp_col = "US_bonds_" + str(decile)
    
    if ret_col in common_df_2.columns and corp_col in common_df_2.columns:
        sub_df = common_df_2[[ret_col, corp_col]].dropna()
        if len(sub_df) > 0:
            corr = sub_df[ret_col].corr(sub_df[corp_col])
            r2 = corr ** 2
        else:
            r2 = None
    else:
        r2 = None

    r_squared_list_2.append({"decile": decile, "r_squared": r2})

r_squared_df_2 = pd.DataFrame(r_squared_list_2)

In [16]:
r_squared_df_2

,decile,r_squared
0,11,0.004609
1,12,0.014647
2,13,0.112480
3,14,0.188833
4,15,0.343133
5,16,0.455347
6,17,0.389103
7,18,0.388928
8,19,0.684873
9,20,0.556141


In [19]:
merged[merged['tau']<1]

,date,cusip,cusip_date,price_eom,tmt,yield,t_yld_pt,ret_eom,year,tr_return,tr_ytm_match,tau,yield_spread,TTM_diff,decile
